In [2]:
import streamlit as st
from langchain.chains import LLMChain, SimpleSequentialChain
import pandas as pd
import sqlite3
import os
from dotenv import load_dotenv
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI
from subs.db_connections import connetc_to_irish_db

load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# Check current working directory.
print("Current Working Directory: ", os.getcwd())


# #llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
db = connetc_to_irish_db()



Current Working Directory:  /Users/saeed/Documents/GitHub/ireland_res_chatbot


In [5]:
system_prefix("chain_1")

'You are an agent designed to interact with a SQL database.\n        Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\n        Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\n        You can order the results by a relevant column to return the most interesting examples in the database.\n        Query for the relevant columns needed to answer the question comprehensively. \n        In cases where calculations or comparisons across multiple sectors (such as IE and NI) are required,\n        ensure to include all necessary columns for these calculations. Prioritize efficiency and relevance in your queries.\n        You have access to tools for interacting with the database.\n        Only use the given tools. Only use the information returned by the tools to construct your final answer.\n\n        You MUST double check

In [6]:
example_selector("chain_1")

SemanticSimilarityExampleSelector(vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1236ba3e0>, k=5, example_keys=None, input_keys=['input'], vectorstore_kwargs=None)

In [7]:
few_shot_prompt("chain_1")

FewShotPromptTemplate(input_variables=['dialect', 'top_k'], example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1236b8b80>, k=5, example_keys=None, input_keys=['input'], vectorstore_kwargs=None), example_prompt=PromptTemplate(input_variables=['input', 'query'], template='User input: {input}\nSQL query: {query}'), suffix='', prefix='You are an agent designed to interact with a SQL database.\n        Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\n        Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\n        You can order the results by a relevant column to return the most interesting examples in the database.\n        Query for the relevant columns needed to answer the question comprehensively. \n        In cases where calculations or compariso

In [10]:
system_message_prompt = SystemMessagePromptTemplate(
        prompt=few_shot_prompt("chain_1")
    )

In [11]:
system_message_prompt

SystemMessagePromptTemplate(prompt=FewShotPromptTemplate(input_variables=['dialect', 'top_k'], example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1236ec670>, k=5, example_keys=None, input_keys=['input'], vectorstore_kwargs=None), example_prompt=PromptTemplate(input_variables=['input', 'query'], template='User input: {input}\nSQL query: {query}'), suffix='', prefix='You are an agent designed to interact with a SQL database.\n        Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\n        Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\n        You can order the results by a relevant column to return the most interesting examples in the database.\n        Query for the relevant columns needed to answer the question comprehensively. \n        In ca

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
input_query={
        "input": "How many years are there",
        "top_k": 5,
        "dialect": "SQLite",
        "agent_scratchpad": [],
    }

ChatPromptTemplate.from_messages(
        [
            system_message_prompt,
            ("human", input_query),
            MessagesPlaceholder("agent_scratchpad"),
        ]
    )

In [3]:
from subs.prompts import *




# Example formatted prompt
prompt_val = invoke_full_prompt(chain_id="chain_1", input_query=
    {
        "input": "How many years are there",
        "top_k": 5,
        "dialect": "SQLite",
        "agent_scratchpad": [],
    })


ValueError: 